<a href="https://colab.research.google.com/github/sbomma1973/AppSearch_DataIngest-csv-Python/blob/master/Elevance_Reference_Case_document_Load_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This program is to illurate how to Chunk and index a word/pdf document into elasticsearch. This includes to store the information not just as text but also as embeddings.

# Step1: Download python libraries to convert docx into json. install and import libaries needed for this demo


In [1]:
!pip install python-docx openai==0.28
!pip install elasticsearch


from docx import Document
from elasticsearch import Elasticsearch
from elasticsearch import Elasticsearch
import pandas as pd
import openai
from getpass import getpass
import openai
import json



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 4.0 MB/s eta 0:00:00


# Step 2: Setup Enviromment variables and Connection Parameters for Elasticsearch and OpenAI - Make sure your elastic search is up and running and you have an OpenAI account. If you are using some other LLM make sure you have connectivity for that


In [2]:
CLOUD_ID=getpass("Enter Cloud ID: ")
CLOUD_USER=getpass("Enter Cloud User:  ")
CLOUD_PASS=getpass("Enter Cloud Password:  ")
elastic_api_key = getpass("Enter Elastic API Key: ")


openai.api_key =getpass("OpenAI API Key: ")

es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=(CLOUD_USER,CLOUD_PASS))

Enter Cloud ID: ··········
Enter Cloud User:  ··········
Enter Cloud Password:  ··········
Enter Elastic API Key: ··········
OpenAI API Key: ··········


In [3]:
es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=(CLOUD_USER,CLOUD_PASS))
document_path = '/content/sample_data/test.docx'
index_name = 'cardiovascular_effective_care'


In [13]:
es.indices.delete(index=index_name, ignore=[400,404])
es.ingest.delete_pipeline(id="docx_pipeline")

pipeline_body = {
    "description": "Ingest pipeline to convert text to embeddings using Elser model",
    "processors": [
        {
            "inference": {
                "model_id": ".elser_model_2",
                "input_output": [
                    {
                        "input_field": "content",
                        "output_field": "content_embedding"
                    }
                ]
            }
        }
    ]
}

# Create the ingest pipeline
response = es.ingest.put_pipeline(id='docx_pipeline', body=pipeline_body)
print("Pipeline creation response:", response)

<ipython-input-13-21b3cb9120c0>:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400,404])


Pipeline creation response: {'acknowledged': True}


In [14]:
index_settings = {
    "settings": {
        "index": {
            "default_pipeline": "docx_pipeline"
        }
    },
    "mappings": {
        "properties": {
            "chunk_id": {
                "type": "long"
            },
            "content": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "content_embedding": {
                "type": "sparse_vector"
            },
            "table": {
                "properties": {
                    "row": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    }
                }
            },
            "table_embedding": {
                "type": "sparse_vector"
            }
        }
    }
}

if not es.indices.exists(index=index_name):
    response = es.indices.create(index=index_name, body=index_settings)
    print("Index creation response:", response)

Index creation response: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'cardiovascular_effective_care'}


## Reference Word Doc Strategy: The word doc, there is text and tables, i have indexed everything into content field in elastic and created embedddings using Elser(elastic Machine learning tool to create sparse embeddings)

You can use any other tool to create embeddings such as (Open AI embeddings)



In [15]:
def extract_text_and_tables_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    tables = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    for table in doc.tables:
        table_data = []
        for row in table.rows:
            row_data = [cell.text for cell in row.cells]
            table_data.append(row_data)
        tables.append(table_data)

    return '\n'.join(full_text), tables


def chunk_text(text, chunk_size=256):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

def chunk_tables(tables, rows_per_chunk=10):
    table_chunks = []
    for table in tables:
        for i in range(0, len(table), rows_per_chunk):
            table_chunks.append(table[i:i + rows_per_chunk])
    return table_chunks

def index_chunks(chunks, index_name, es_client):

  documents = []

  for i, chunk in enumerate(text_chunks):
        doc = {
            'content': chunk,
            'chunk_id': i
        }
        documents.append(doc)
        #print (doc)

  for i, table_chunk in enumerate(table_chunks):
    # Ensure each item in table_chunk is a string
    table_chunk_str = ' '.join(str(item) for item in table_chunk)
    doc = {
        'content': table_chunk_str,  # Concatenate rows into a single string
        'chunk_id': len(text_chunks) + i
    }
    documents.append(doc)

  for doc in documents:
    es.index(index=index_name, body=doc)


## Chucking Strategy - I chunked the document you provided into 256 bytes per chunk - You can have different chunking strategys based on how granular or overlapping your search to be


In [16]:
text, tables = extract_text_and_tables_from_docx(document_path)
text_chunks = chunk_text(text, chunk_size=256)
table_chunks = chunk_tables(tables, rows_per_chunk=100)

#print(text)
#print(tables)
chunks = chunk_text(text)
index_chunks(chunks, index_name, es)

In [17]:
!pip install tiktoken openai==0.28

import openai
import tiktoken

def truncate_text(text, max_tokens):
    tokens = text.split()
    if len(tokens) <= max_tokens:
        return text, len(tokens)

    return ' '.join(tokens[:max_tokens]), len(tokens)

def encoding_token_count(string: str, encoding_model: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    return len(encoding.encode(string))

def chat_gpt(prompt, model="gpt-3.5-turbo", max_tokens=1024, max_context_tokens=4000, safety_margin=5):
    # Truncate the prompt content to fit within the model's context length
    truncated_prompt, word_count = truncate_text(prompt, max_context_tokens - max_tokens - safety_margin)
    openai_token_count = encoding_token_count(prompt, model)
    #print(f"word_count = {word_count}, openai_token_count = {openai_token_count}")

    # Updated way to call the ChatCompletion API in openai>=1.0.0
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an Elevance's Claims Expert and understand Medical Terminology"},
            {"role": "user", "content": truncated_prompt}
        ]
    )

    # Access the message content from the response
    return response.choices[0].message.content, word_count, openai_token_count
    return response["choices"][0]["message"]["content"], word_count, openai_token_count

def search1(query2):
    response = es.search(
        index="cardiovascular_effective_care",
        size=1,
        query={
            "text_expansion": {
                "content_embedding": {
                    "model_id": ".elser_model_2",
                    "model_text": query2
                }
            }
        },
    )
    content = []
    for hit in response["hits"]["hits"]:
        content = hit["_source"]["content"]

        print('Results from elastic:', content)
    return content


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.8 MB/s eta 0:00:00


## OPenAI connector - with OpenAI API key with gpt-3.5-turbo (Cheapest), You could use your own LLM (On prem)
##Other techniques you can use to save cost is cache the responses for similar questions


In [18]:
from typing_extensions import ContextManager
def test():
    #query_text = 'Show me some developers who have also have business analysis'
    query1 = 'what is Cholinesterase inhibitor?'
    negResponse = "I don't know"

    print (query1)
    content = search1(query1)
    print('resp: ', content)

    prompt = f"Answer this question in summarized bullets: {query1}\n Provide information based on {content}\nIf the answer is not contained in the supplied doc reply '{negResponse}' and nothing else"
        #print (prompt)

    answer, word_count, openai_token_count = chat_gpt(prompt)
        #print(f"Response {i + 1}:")
    print(f"----------------------------\n\n\n")
    print(f"Response from LLM: {content}\nAnswer: {answer}")
    #print(f"Word Count: {word_count}")
    #print(f"OpenAI Token Count: {openai_token_count}")
test()


what is Cholinesterase inhibitor?
Results from elastic: ['Description', 'Prescription', 'Prescription', 'Prescription'] ['Cholinesterase inhibitors', 'Donepezil', 'Galantamine', 'Rivastigmine '] ['Miscellaneous central nervous system agents', 'Memantine', 'Memantine', 'Memantine'] ['Dementia combinations', 'Donepezil-memantine', 'Donepezil-memantine', 'Donepezil-memantine']
resp:  ['Description', 'Prescription', 'Prescription', 'Prescription'] ['Cholinesterase inhibitors', 'Donepezil', 'Galantamine', 'Rivastigmine '] ['Miscellaneous central nervous system agents', 'Memantine', 'Memantine', 'Memantine'] ['Dementia combinations', 'Donepezil-memantine', 'Donepezil-memantine', 'Donepezil-memantine']
----------------------------



Response from LLM: ['Description', 'Prescription', 'Prescription', 'Prescription'] ['Cholinesterase inhibitors', 'Donepezil', 'Galantamine', 'Rivastigmine '] ['Miscellaneous central nervous system agents', 'Memantine', 'Memantine', 'Memantine'] ['Dementia combina